In [1]:
# Importing all the libraries
import pandas as pd
import os
import warnings
warnings.filterwarnings('ignore')

# Library that we have created
import Utilities as ut

# Loading Data

### Premier League

Getting all the stats of Premier Leagues Season for 5 years, between 2014-2015 and 2018-2019

In [2]:
# Aggregate all the files in a single one
dataset_position_PL = "dataset/Premier League/"

# Identify all the datatests within the directory
file_list_PL = os.listdir(dataset_position_PL) 

for element in file_list_PL:
    data = ut.open_csv(element,dataset_position_PL)

    # This function allows us to create a variable name with the name of the dataset
    globals()["PL_" + str(element[:-4])] = data
    
    #  Drop the columns that we don't need
    globals()["PL_" + str(element[:-4])] = globals()["PL_" + str(element[:-4])].drop(['odds_ft_home_team_win','odds_ft_draw', 'odds_ft_away_team_win', 'odds_ft_over15', 'odds_ft_over25', 'odds_ft_over35', 'odds_ft_over45', 'odds_btts_yes', 'odds_btts_no', 'stadium_name',], axis = 1)#We create a Dictionary with all the teams per year
Teams_per_year = {}

# Storing which team plays in PL each year:
for element in file_list_PL: 
    team = ut.PL_team(element,dataset_position_PL)
    Teams_per_year.update({element:team})
    
# Create a list with all the years   
year_list = []
for k in Teams_per_year.keys():
    k = k[:-4]
    year_list.append(k)
year_list.sort()

### FIFA

Getting the Squad rating, as known as overall weighted, for each team of Premier League, for 5 Seasons, between 2014-2015 and 2018-2019. Every overall is calculated on the popular FIFA videogames.

#### The overall algorithm we used is:

<img src ="https://i.ibb.co/VJ4Kp8F/Screen-Shot-2020-03-22-at-5-36-01-PM.png" width = "500px">

<b>Note: </b>
<ol>
<li>The CF is calculted only if the overall of the player is bigger than the Avg of all the players of the team</li>
<li>If the player is a substitute, and the note one is true, the "weight" divided in half  </li>
 </ol>

In [3]:
dataset_position_FIFA = "dataset/Fifa/"

# Identify all the datatests within the directory
file_list_FIFA = os.listdir(dataset_position_FIFA) 

FIFA_dict = {}
for element in file_list_FIFA:
    
    # The year name of the file
    name = element[5:14]
    
    # Open and create the dataset
    data = ut.open_csv(element,dataset_position_FIFA)
    
    # This function allows us to create a variable name with the name of the dataset
    globals()["FIFA_" + str(name)] = data 
    FIFA_dict.update({str(name) : globals()["FIFA_" + str(name)]})

# Data Cleaning

In [4]:
# Create a empty dataframe
agg_PL  = pd.DataFrame(None)

for k in year_list:
    
    # Set up all the dataframes using the class 
    globals()["PL_" + str(k)] = ut.Set_up_data(Teams_per_year, globals()["FIFA_" + str(k)], globals()["FIFA_" + str(k)].name ,globals()["PL_" + str(k)]).add_to_PL()
    
    # Append all the PL dataframes
    agg_PL =  agg_PL.append(globals()["PL_" + str(k)])

In [5]:
# Keep only "core data" to build a dataframe step by step
col = ["timestamp", 'date_GMT', 'home_team_name','away_team_name','home_team_goal_count','away_team_goal_count','Result', 'home_team_overall',
       'away_team_overall','home_team_corner_count', 'away_team_corner_count',
       'home_team_yellow_cards', 'home_team_red_cards',
       'away_team_yellow_cards', 'away_team_red_cards',
       'home_team_shots','away_team_shots', 'home_team_shots_on_target',
       'away_team_shots_on_target', 'home_team_fouls', 'away_team_fouls']

for i in year_list:
    globals()["core_data_" + str(i)] =  globals()["PL_" + str(i)][col]
    
    # Rename the columns
    globals()["core_data_" + str(i)] = globals()["core_data_" + str(i)].rename(columns={
                                                     "home_team_goal_count": "Hteam_FT",
                                                     "away_team_goal_count": "Ateam_FT",
                                                     "home_team_overall": "Hteam_ov",
                                                     "away_team_overall": "Ateam_ov",
                                                     "home_team_corner_count": "Hteam_CC",
                                                     "away_team_corner_count": "Ateam_CC",
                                                     "home_team_yellow_cards": "Hteam_YC",
                                                     'home_team_red_cards':"Hteam_RC",
                                                     'away_team_yellow_cards':"Ateam_YC" ,
                                                     'away_team_red_cards': "Ateam_RC",
                                                     'home_team_shots' : "Hteam_SH",
                                                     'home_team_shots_on_target':"Hteam_SoT",
                                                     'away_team_shots': "Ateam_SH",
                                                     'away_team_shots_on_target': "Ateam_SoT",
                                                     'home_team_fouls': "Hteam_F",
                                                     'away_team_fouls': "Ateam_F"})
    
core_data_2018_2019.head(3)

,timestamp,date_GMT,home_team_name,away_team_name,Hteam_FT,Ateam_FT,Result,Hteam_ov,Ateam_ov,Hteam_CC,...,Hteam_YC,Hteam_RC,Ateam_YC,Ateam_RC,Hteam_SH,Ateam_SH,Hteam_SoT,Ateam_SoT,Hteam_F,Ateam_F
0,1533927600,10-08-2018,Manchester United,Leicester City,2,1,H,84.413580,79.216049,2,...,2,0,1,0,9,14,7,5,11,7
1,1533987000,11-08-2018,Newcastle United,Tottenham Hotspur,1,2,A,76.574074,84.527778,3,...,2,0,2,0,16,17,3,7,10,12
2,1533996000,11-08-2018,Bournemouth,Cardiff City,2,0,H,76.969136,73.757716,7,...,1,0,1,0,9,11,4,2,11,7


# Feature engineering

### - Features of each matchweek (updated before the start of the game)
We calculate the stats (done and concedeed) of a variety features for every teams (Home and Away teams).
<br> Every feature is calculated before the start of the game. The features calculated are:
<ul>
<li>Number of <b>Goals</b> (Saved as G)</li>
<li>Number of <b>Yellow Cards</b> (Saved as YC)</li>
<li>Number of <b>Reds Cards</b> (Saved as RC)</li>
<li>Number of <b>Shots</b> (Saved as SH)</li>
<li>Number of <b>Shots on target</b> (Saved as SoT)</li>
<li>Number of <b>Corners</b> (Saved as CC)</li>
<li>Number of <b>Fouls</b> (Saved as F)</li>
</ul>

In [6]:
# Gets the specific feature doneby teams at specific matchweek

def feature_done(df, feat1, feat2):
    # Create a dictionary with team names as keys
    teams = {}
    for i in list(set(df['home_team_name'])):
        teams[i] = []
    
    # The value corresponds to keys is a list containing the match location.
    for i in range(len(df)):
        Home_team_feature = df.iloc[i][feat1] # Column 1
        Away_team_feature = df.iloc[i][feat2] # Column 2
        
        # Add the feature to the respective team
        teams[df.iloc[i].home_team_name].append(Home_team_feature)
        teams[df.iloc[i].away_team_name].append(Away_team_feature)
    
    # Create a dataframe for feature done where rows are teams and cols are matchweek.
    feature = pd.DataFrame(data = teams, index = [i for i in range(1,39)]).T
    
    # Add a column zero, in order to make computations
    feature[0] = 0
    # Aggregate to get uptil that point
    for i in range(2,39):
        # At the features is added what happened the previous weeks
        feature[i] += feature[i - 1]
    return feature

In [7]:
# Gets the features done aggregated arranged by teams and matchweek
def feature_conceded(df, feat1, feat2):
    # Create a dictionary with team names as keys
    teams = {}
    for i in list(set(df['home_team_name'])):
        teams[i] = []
    
    # The value corresponds to keys is a list containing the match location.
    for i in range(len(df)):
        Home_team_feature = df.iloc[i][feat1] # Column 1
        Away_team_feature = df.iloc[i][feat2] # Column 2
        
        # Add the feature to the opponent team 
        teams[df.iloc[i].home_team_name].append(Away_team_feature)
        teams[df.iloc[i].away_team_name].append(Home_team_feature)
    
    # Create a dataframe for features done where rows are teams and cols are matchweek.
    feature = pd.DataFrame(data = teams, index = [i for i in range(1,39)]).T
    
    # Add a column zero, in order to make computations
    feature[0] = 0
    # Aggregate to get uptil that point
    for i in range(2, 39):
        # At the features is added what happened the previous weeks
        feature[i] +=  feature[i - 1]
    return feature

In [8]:
def get_step(df, f1, f2, H_1, A_1, H_2, A_2):
    feature_Conceded = feature_conceded(df, f1, f2)
    feature_Done = feature_done(df, f1, f2)
   
    counter = 0
    
    Home_team_f_done = []
    Away_team_f_done = []
    Home_team_f_conceded = []
    Away_team_f_conceded = []

    for i in range(380):
        # Get the name of home team
        home = df.iloc[i].home_team_name
        # Get the name of away team
        away = df.iloc[i].away_team_name
        
        # Add to the Home team, in the right position the features
        Home_team_f_done.append(feature_Done.loc[home][counter])
        Home_team_f_conceded.append(feature_Conceded.loc[home][counter])
        
        # Add to the Away team, in the right position the features
        Away_team_f_conceded.append(feature_Conceded.loc[away][counter])
        Away_team_f_done.append(feature_Done.loc[away][counter])
        
        # New Match week 
        if ((i + 1)% 10) == 0:
            counter += 1
    
    # Add the features to the Dataframe
    df[H_1] = Home_team_f_done
    df[A_1] = Away_team_f_done
    df[H_2] = Home_team_f_conceded
    df[A_2] = Away_team_f_conceded
    
    return df

In [9]:
# Doing the process for a large quantity of features
for i in year_list:
    df_2018 = get_step(df_2018,"Hteam_FT","Ateam_FT",'Hteam_G_got','Ateam_G_got','Hteam_G_opp','Ateam_G_opp')
    globals()["core_data_" + str(i)] =  get_step(globals()["core_data_" + str(i)],"Hteam_YC","Ateam_YC","Hteam_YC_got","Ateam_YC_got","Hteam_YC_opp","Ateam_YC_opp")
    globals()["core_data_" + str(i)] =  get_step(globals()["core_data_" + str(i)],"Hteam_RC","Ateam_RC","Hteam_RC_got","Ateam_RC_got","Hteam_RC_opp","Ateam_RC_opp")
    globals()["core_data_" + str(i)] =  get_step(globals()["core_data_" + str(i)],"Hteam_CC","Ateam_CC","Hteam_CC_got","Ateam_CC_got","Hteam_CC_opp","Ateam_CC_opp")
    globals()["core_data_" + str(i)] =  get_step(globals()["core_data_" + str(i)],"Hteam_F","Ateam_F","Hteam_F_got","Ateam_F_got","Hteam_F_opp","Ateam_F_opp")
    globals()["core_data_" + str(i)] =  get_step(globals()["core_data_" + str(i)],"Hteam_SH","Ateam_SH","Hteam_SH_got","Ateam_SH_got","Hteam_SH_opp","Ateam_SH_opp")
    globals()["core_data_" + str(i)] =  get_step(globals()["core_data_" + str(i)],"Hteam_SoT","Ateam_SoT","Hteam_SoT_got","Ateam_SoT_got","Hteam_SoT_opp","Ateam_SoT_opp")

core_data_2018_2019.head(3)

,timestamp,date_GMT,home_team_name,away_team_name,Hteam_FT,Ateam_FT,Result,Hteam_ov,Ateam_ov,Hteam_CC,...,Hteam_F_opp,Ateam_F_opp,Hteam_SH_got,Ateam_SH_got,Hteam_SH_opp,Ateam_SH_opp,Hteam_SoT_got,Ateam_SoT_got,Hteam_SoT_opp,Ateam_SoT_opp
0,1533927600,10-08-2018,Manchester United,Leicester City,2,1,H,84.413580,79.216049,2,...,0,0,0,0,0,0,0,0,0,0
1,1533987000,11-08-2018,Newcastle United,Tottenham Hotspur,1,2,A,76.574074,84.527778,3,...,0,0,0,0,0,0,0,0,0,0
2,1533996000,11-08-2018,Bournemouth,Cardiff City,2,0,H,76.969136,73.757716,7,...,0,0,0,0,0,0,0,0,0,0


### - Winning/Losing Streak
Calculating the Winning/Losing Streak for every match, for both teams (Home and Away)

In [10]:
def match_res(df):
    # Create a dictionary with team names as keys
    teams = {}
    for i in list(set(df['home_team_name'])):
        teams[i] = []

    # The value corresponds to keys is a list containing the match result
    for i in range(len(df)):
        if df.iloc[i].Result == 'H':
            # Append "W" value to the Home Team and "L" to teh Away team
            teams[df.iloc[i].home_team_name].append('W')
            teams[df.iloc[i].away_team_name].append('L')
        elif df.iloc[i].Result == 'A':
            # Append "L" value to the Home Team and "W" to the Away team
            teams[df.iloc[i].away_team_name].append('W')
            teams[df.iloc[i].home_team_name].append('L')
        else:
            # Append "D" value to both teams
            teams[df.iloc[i].away_team_name].append('D')
            teams[df.iloc[i].home_team_name].append('D')
            
    return pd.DataFrame(data = teams, index = [i for i in range(1,39)]).T


def form_stats(df, num):
    form = match_res(df)
    
    form_final = form.copy()
    
    for i in range(num, 39):
        form_final[i] = ''
        counter = 0
        while counter < num:
            form_final[i] += form[i - counter]
            counter += 1           
    return form_final

def add_form(df,num):
    form = form_stats(df, num)
    
    # There are not previous result in the first 10 matches
    h = ['M' for i in range(num * 10)]  
    a = ['M' for i in range(num * 10)]
    
    counter = num
    for i in range((num*10), 380):
        ht = df.iloc[i].home_team_name
        at = df.iloc[i].away_team_name
        
        past = form.loc[ht][counter]               # get past n results
        h.append(past[num - 1])                    # 0 index is most recent
        
        past = form.loc[at][counter]               # get past n results.
        a.append(past[num - 1])                   # 0 index is most recent
        
        if ((i + 1)% 10) == 0:
            counter += 1

    df['Hteam_Bef' + str(num)] = h                 
    df['Ateam_Bef' + str(num)] = a
 
    return df

def add_form_df(df):
    df = add_form(df,1) # 1 previous game
    df = add_form(df,2) # 2 previous games
    df = add_form(df,3) # 3 previous games
    df = add_form(df,4) # 4 previous games
    df = add_form(df,5) # 5 previous games
    return df 

for i in year_list:
    globals()["core_data_" + str(i)] =  add_form_df(globals()["core_data_" + str(i)])

core_data_2018_2019.head(3) 

,timestamp,date_GMT,home_team_name,away_team_name,Hteam_FT,Ateam_FT,Result,Hteam_ov,Ateam_ov,Hteam_CC,...,Hteam_Bef1,Ateam_Bef1,Hteam_Bef2,Ateam_Bef2,Hteam_Bef3,Ateam_Bef3,Hteam_Bef4,Ateam_Bef4,Hteam_Bef5,Ateam_Bef5
0,1533927600,10-08-2018,Manchester United,Leicester City,2,1,H,84.413580,79.216049,2,...,M,M,M,M,M,M,M,M,M,M
1,1533987000,11-08-2018,Newcastle United,Tottenham Hotspur,1,2,A,76.574074,84.527778,3,...,M,M,M,M,M,M,M,M,M,M
2,1533996000,11-08-2018,Bournemouth,Cardiff City,2,0,H,76.969136,73.757716,7,...,M,M,M,M,M,M,M,M,M,M


### - Appending the Match week number
Adding to each games the relative Match Week Number

In [11]:
for i in year_list:
    globals()["core_data_" + str(i)] =  ut.match_week(globals()["core_data_" + str(i)])
        
core_data_2018_2019.head(3)

,timestamp,date_GMT,home_team_name,away_team_name,Hteam_FT,Ateam_FT,Result,Hteam_ov,Ateam_ov,Hteam_CC,...,Ateam_Bef1,Hteam_Bef2,Ateam_Bef2,Hteam_Bef3,Ateam_Bef3,Hteam_Bef4,Ateam_Bef4,Hteam_Bef5,Ateam_Bef5,MW
0,1533927600,10-08-2018,Manchester United,Leicester City,2,1,H,84.413580,79.216049,2,...,M,M,M,M,M,M,M,M,M,1
1,1533987000,11-08-2018,Newcastle United,Tottenham Hotspur,1,2,A,76.574074,84.527778,3,...,M,M,M,M,M,M,M,M,M,1
2,1533996000,11-08-2018,Bournemouth,Cardiff City,2,0,H,76.969136,73.757716,7,...,M,M,M,M,M,M,M,M,M,1


### - A unique dataframe
Concatenating all the dataframes (5 seasons dataframe)

In [12]:
core_data_list = []
for i in year_list:
    core_data_list.append (globals()["core_data_" + str(i)])
    
# Create a unique dataset    
core_data = pd.concat(core_data_list, ignore_index=True)
data = core_data.copy() 
core_data.head(3)

,timestamp,date_GMT,home_team_name,away_team_name,Hteam_FT,Ateam_FT,Result,Hteam_ov,Ateam_ov,Hteam_CC,...,Ateam_Bef1,Hteam_Bef2,Ateam_Bef2,Hteam_Bef3,Ateam_Bef3,Hteam_Bef4,Ateam_Bef4,Hteam_Bef5,Ateam_Bef5,MW
0,1408189500,16-08-2014,Manchester United,Swansea City,1,2,A,82.629630,75.617284,4,...,M,M,M,M,M,M,M,M,M,1
1,1408197600,16-08-2014,Leicester City,Everton,2,2,D,71.694444,79.413580,3,...,M,M,M,M,M,M,M,M,M,1
2,1408197600,16-08-2014,Stoke City,Aston Villa,0,1,A,76.191358,75.388889,2,...,M,M,M,M,M,M,M,M,M,1


### -  Averaging Features
Doing the mean of each features by the match week number 

In [13]:
features = ['Hteam_G_got', 'Ateam_G_got', 'Hteam_G_opp' , 'Ateam_G_opp',
            'Hteam_YC_got', 'Ateam_YC_got', 'Hteam_YC_opp', 'Ateam_YC_opp',
            'Hteam_RC_got', 'Ateam_RC_got', 'Hteam_RC_opp', 'Ateam_RC_opp',
            'Hteam_CC_got', 'Ateam_CC_got', 'Hteam_CC_opp', 'Ateam_CC_opp',
            'Hteam_F_got', 'Ateam_F_got', 'Hteam_F_opp', 'Ateam_F_opp',
            'Hteam_SH_got', 'Ateam_SH_got', 'Hteam_SH_opp', 'Ateam_SH_opp',
            'Hteam_SoT_got', 'Ateam_SoT_got','Hteam_SoT_opp','Ateam_SoT_opp']

for mw in list(data.MW):
    if mw > 1:
        for name in features:
            core_data[name + "_M" ] = core_data[name]/ (mw -1)
core_data.head(3)

,timestamp,date_GMT,home_team_name,away_team_name,Hteam_FT,Ateam_FT,Result,Hteam_ov,Ateam_ov,Hteam_CC,...,Hteam_F_opp_M,Ateam_F_opp_M,Hteam_SH_got_M,Ateam_SH_got_M,Hteam_SH_opp_M,Ateam_SH_opp_M,Hteam_SoT_got_M,Ateam_SoT_got_M,Hteam_SoT_opp_M,Ateam_SoT_opp_M
0,1408189500,16-08-2014,Manchester United,Swansea City,1,2,A,82.629630,75.617284,4,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1408197600,16-08-2014,Leicester City,Everton,2,2,D,71.694444,79.413580,3,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1408197600,16-08-2014,Stoke City,Aston Villa,0,1,A,76.191358,75.388889,2,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [15]:
pd.set_option('display.max_columns', None)  
core_data.tail()

,timestamp,date_GMT,home_team_name,away_team_name,Hteam_FT,Ateam_FT,Result,Hteam_ov,Ateam_ov,Hteam_CC,Ateam_CC,Hteam_YC,Hteam_RC,Ateam_YC,Ateam_RC,Hteam_SH,Ateam_SH,Hteam_SoT,Ateam_SoT,Hteam_F,Ateam_F,Hteam_G_got,Ateam_G_got,Hteam_G_opp,Ateam_G_opp,Hteam_YC_got,Ateam_YC_got,Hteam_YC_opp,Ateam_YC_opp,Hteam_RC_got,Ateam_RC_got,Hteam_RC_opp,Ateam_RC_opp,Hteam_CC_got,Ateam_CC_got,Hteam_CC_opp,Ateam_CC_opp,Hteam_F_got,Ateam_F_got,Hteam_F_opp,Ateam_F_opp,Hteam_SH_got,Ateam_SH_got,Hteam_SH_opp,Ateam_SH_opp,Hteam_SoT_got,Ateam_SoT_got,Hteam_SoT_opp,Ateam_SoT_opp,Hteam_Bef1,Ateam_Bef1,Hteam_Bef2,Ateam_Bef2,Hteam_Bef3,Ateam_Bef3,Hteam_Bef4,Ateam_Bef4,Hteam_Bef5,Ateam_Bef5,MW,Hteam_G_got_M,Ateam_G_got_M,Hteam_G_opp_M,Ateam_G_opp_M,Hteam_YC_got_M,Ateam_YC_got_M,Hteam_YC_opp_M,Ateam_YC_opp_M,Hteam_RC_got_M,Ateam_RC_got_M,Hteam_RC_opp_M,Ateam_RC_opp_M,Hteam_CC_got_M,Ateam_CC_got_M,Hteam_CC_opp_M,Ateam_CC_opp_M,Hteam_F_got_M,Ateam_F_got_M,Hteam_F_opp_M,Ateam_F_opp_M,Hteam_SH_got_M,Ateam_SH_got_M,Hteam_SH_opp_M,Ateam_SH_opp_M,Hteam_SoT_got_M,Ateam_SoT_got_M,Hteam_SoT_opp_M,Ateam_SoT_opp_M
1895,1557669600,12-05-2019,Manchester United,Cardiff City,0,2,A,84.413580,73.757716,11,2,3,0,3,0,27,14,11,5,8,6,65,32,52,69,72,63,64,46,4,1,1,3,189,166,184,259,412,363,391,389,521,440,518,575,248,156,209,239,D,L,D,L,L,L,L,W,W,L,38,1.756757,0.864865,1.405405,1.864865,1.945946,1.702703,1.729730,1.243243,0.108108,0.027027,0.027027,0.081081,5.108108,4.486486,4.972973,7.000000,11.135135,9.810811,10.567568,10.513514,14.081081,11.891892,14.000000,15.540541,6.702703,4.216216,5.648649,6.459459
1896,1557669600,12-05-2019,Liverpool,Wolverhampton Wanderers,2,0,H,83.740741,77.117284,4,1,0,0,2,0,14,8,6,3,4,11,87,47,22,44,39,70,46,76,2,1,1,4,245,194,125,186,307,383,344,321,604,510,342,484,263,191,134,179,W,W,W,W,W,W,W,D,W,L,38,2.351351,1.270270,0.594595,1.189189,1.054054,1.891892,1.243243,2.054054,0.054054,0.027027,0.027027,0.108108,6.621622,5.243243,3.378378,5.027027,8.297297,10.351351,9.297297,8.675676,16.324324,13.783784,9.243243,13.081081,7.108108,5.162162,3.621622,4.837838
1897,1557669600,12-05-2019,Watford,West Ham United,1,4,A,77.589506,79.472222,7,2,1,1,0,0,18,15,9,8,9,11,51,48,55,54,77,59,50,57,3,1,3,0,170,186,208,200,416,311,383,385,452,468,505,554,185,183,209,233,L,W,L,W,D,D,W,L,L,L,38,1.378378,1.297297,1.486486,1.459459,2.081081,1.594595,1.351351,1.540541,0.081081,0.027027,0.081081,0.000000,4.594595,5.027027,5.621622,5.405405,11.243243,8.405405,10.351351,10.405405,12.216216,12.648649,13.648649,14.972973,5.000000,4.945946,5.648649,6.297297
1898,1557669600,12-05-2019,Fulham,Newcastle United,0,4,A,77.135802,76.574074,5,5,1,0,0,0,16,15,3,7,3,7,34,38,77,48,69,57,43,57,2,2,0,1,156,176,221,226,401,406,335,334,458,466,606,509,178,177,259,188,L,L,W,D,W,W,W,W,L,L,38,0.918919,1.027027,2.081081,1.297297,1.864865,1.540541,1.162162,1.540541,0.054054,0.054054,0.000000,0.027027,4.216216,4.756757,5.972973,6.108108,10.837838,10.972973,9.054054,9.027027,12.378378,12.594595,16.378378,13.756757,4.810811,4.783784,7.000000,5.081081
1899,1557669600,12-05-2019,Brighton & Hove Albion,Manchester City,1,4,A,76.641975,85.975309,2,6,0,0,0,0,7,22,3,11,12,8,34,91,56,22,61,44,52,54,4,1,2,2,154,292,210,80,428,309,289,287,402,698,580,270,145,288,199,114,D,W,D,W,L,W,D,W,L,W,38,0.918919,2.459459,1.513514,0.594595,1.648649,1.189189,1.405405,1.459459,0.108108,0.027027,0.054054,0.054054,4.162162,7.891892,5.675676,2.162162,11.567568,8.351351,7.810811,7.756757,10.864865,18.864865,15.675676,7.297297,3.918919,7.783784,5.378378,3.081081


### -  Make Differences 
For each Team (Home and Away) we make the difference for each feature done and conceeded

In [16]:
# Get Goal Difference 
core_data["Diff_HT_G_M"] = core_data["Hteam_G_got_M"] - core_data["Hteam_G_opp_M"]
core_data["Diff_AT_G_M"] = core_data["Ateam_G_got_M"] - core_data["Ateam_G_opp_M"]

# Get Yellow Cards Difference
core_data["Diff_HT_YC_M"] = core_data["Hteam_YC_got_M"] - core_data["Hteam_YC_opp_M"]
core_data["Diff_AT_YC_M"] = core_data["Ateam_YC_got_M"] - core_data["Ateam_YC_opp_M"]

# Get Red Cards Difference
core_data["Diff_HT_RC_M"] = core_data["Hteam_RC_got_M"] - core_data["Hteam_RC_opp_M"]
core_data["Diff_AT_RC_M"] = core_data["Ateam_RC_got_M"] - core_data["Ateam_RC_opp_M"]

# Get Number of corners difference
core_data["Diff_HT_CC_M"] = core_data["Hteam_CC_got_M"] - core_data["Hteam_CC_opp_M"]
core_data["Diff_AT_CC_M"] = core_data["Ateam_CC_got_M"] - core_data["Ateam_CC_opp_M"]

# Get Number of fouls difference
core_data["Diff_HT_F_M"] = core_data["Hteam_F_got_M"] - core_data["Hteam_F_opp_M"]
core_data["Diff_AT_F_M"] = core_data["Ateam_F_got_M"] - core_data["Ateam_F_opp_M"]

# Get Number of Total shots difference
core_data["Diff_HT_SH_M"] = core_data["Hteam_SH_got_M"] - core_data["Hteam_SH_opp_M"]
core_data["Diff_AT_SH_M"] = core_data["Ateam_SH_got_M"] - core_data["Ateam_SH_opp_M"]

# Get Number of Target shots difference
core_data["Diff_HT_SoT_M"] = core_data["Hteam_SoT_got_M"] - core_data["Hteam_SoT_opp_M"]
core_data["Diff_AT_SoT_M"] = core_data["Ateam_SoT_got_M"] - core_data["Ateam_SoT_opp_M"]

# Get Difference in team overall
core_data['Diff_ov'] = core_data['Hteam_ov'] - core_data['Ateam_ov']

core_data.head(3)

,timestamp,date_GMT,home_team_name,away_team_name,Hteam_FT,Ateam_FT,Result,Hteam_ov,Ateam_ov,Hteam_CC,Ateam_CC,Hteam_YC,Hteam_RC,Ateam_YC,Ateam_RC,Hteam_SH,Ateam_SH,Hteam_SoT,Ateam_SoT,Hteam_F,Ateam_F,Hteam_G_got,Ateam_G_got,Hteam_G_opp,Ateam_G_opp,Hteam_YC_got,Ateam_YC_got,Hteam_YC_opp,Ateam_YC_opp,Hteam_RC_got,Ateam_RC_got,Hteam_RC_opp,Ateam_RC_opp,Hteam_CC_got,Ateam_CC_got,Hteam_CC_opp,Ateam_CC_opp,Hteam_F_got,Ateam_F_got,Hteam_F_opp,Ateam_F_opp,Hteam_SH_got,Ateam_SH_got,Hteam_SH_opp,Ateam_SH_opp,Hteam_SoT_got,Ateam_SoT_got,Hteam_SoT_opp,Ateam_SoT_opp,Hteam_Bef1,Ateam_Bef1,Hteam_Bef2,Ateam_Bef2,Hteam_Bef3,Ateam_Bef3,Hteam_Bef4,Ateam_Bef4,Hteam_Bef5,Ateam_Bef5,MW,Hteam_G_got_M,Ateam_G_got_M,Hteam_G_opp_M,Ateam_G_opp_M,Hteam_YC_got_M,Ateam_YC_got_M,Hteam_YC_opp_M,Ateam_YC_opp_M,Hteam_RC_got_M,Ateam_RC_got_M,Hteam_RC_opp_M,Ateam_RC_opp_M,Hteam_CC_got_M,Ateam_CC_got_M,Hteam_CC_opp_M,Ateam_CC_opp_M,Hteam_F_got_M,Ateam_F_got_M,Hteam_F_opp_M,Ateam_F_opp_M,Hteam_SH_got_M,Ateam_SH_got_M,Hteam_SH_opp_M,Ateam_SH_opp_M,Hteam_SoT_got_M,Ateam_SoT_got_M,Hteam_SoT_opp_M,Ateam_SoT_opp_M,Diff_HT_G_M,Diff_AT_G_M,Diff_HT_YC_M,Diff_AT_YC_M,Diff_HT_RC_M,Diff_AT_RC_M,Diff_HT_CC_M,Diff_AT_CC_M,Diff_HT_F_M,Diff_AT_F_M,Diff_HT_SH_M,Diff_AT_SH_M,Diff_HT_SoT_M,Diff_AT_SoT_M,Diff_ov
0,1408189500,16-08-2014,Manchester United,Swansea City,1,2,A,82.629630,75.617284,4,0,2,0,4,0,5,3,3,3,14,20,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,M,M,M,M,M,M,M,M,M,M,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,7.012346
1,1408197600,16-08-2014,Leicester City,Everton,2,2,D,71.694444,79.413580,3,6,1,0,1,0,8,7,2,4,15,11,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,M,M,M,M,M,M,M,M,M,M,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-7.719136
2,1408197600,16-08-2014,Stoke City,Aston Villa,0,1,A,76.191358,75.388889,2,8,0,0,3,0,5,3,3,0,13,7,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,M,M,M,M,M,M,M,M,M,M,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.802469


In [17]:
drop_col = ['Hteam_ov','Ateam_ov', 'Hteam_FT', 'Ateam_FT','Hteam_CC','Ateam_CC','Hteam_YC', 'Hteam_RC', 'Ateam_YC', 'Ateam_RC','Hteam_SH', 'Ateam_SH', 'Hteam_SoT', 'Ateam_SoT','Hteam_F', 'Ateam_F', 'Hteam_G_got', 'Ateam_G_got','Hteam_G_opp', 'Ateam_G_opp', 'Hteam_YC_got', 'Ateam_YC_got','Hteam_YC_opp', 'Ateam_YC_opp', 'Hteam_RC_got', 'Ateam_RC_got','Hteam_RC_opp', 'Ateam_RC_opp', 'Hteam_CC_got', 'Ateam_CC_got', 'Hteam_CC_opp', 'Ateam_CC_opp', 'Hteam_F_got', 'Ateam_F_got','Hteam_F_opp', 'Ateam_F_opp', 'Hteam_SH_got', 'Ateam_SH_got','Hteam_SH_opp', 'Ateam_SH_opp', 'Hteam_SoT_got', 'Ateam_SoT_got','Hteam_SoT_opp', 'Ateam_SoT_opp']

# Drop useless columns
core_data = core_data.drop(drop_col,1)

# Drop the rows of the first 5 weeks, because we don't know what happened before
core_data = core_data[core_data.MW > 5]

### - Latitude and Longitude of stadium
Adding to each game the Latitude and Longitude for the place of the game

In [18]:
stadium = ut.open_csv("stadiums-coordinates.csv", "dataset/Other/")

# Add to the dataset the latitude and longitude of each match
H_Team_Lat = []
H_Team_Lon = []

for home_team in core_data["home_team_name"]:
    for team, lat, lon in zip (stadium["Team"],stadium["Latitude"], stadium["Longitude"]):
        if home_team == team.strip():
    
            H_Team_Lat.append(lat)
            H_Team_Lon.append(lon)
    
# Adding columns to the dataframe
core_data["Hteam_Lat"] = H_Team_Lat
core_data["Hteam_Lon"] = H_Team_Lon
core_data.head(3)

,timestamp,date_GMT,home_team_name,away_team_name,Result,Hteam_Bef1,Ateam_Bef1,Hteam_Bef2,Ateam_Bef2,Hteam_Bef3,Ateam_Bef3,Hteam_Bef4,Ateam_Bef4,Hteam_Bef5,Ateam_Bef5,MW,Hteam_G_got_M,Ateam_G_got_M,Hteam_G_opp_M,Ateam_G_opp_M,Hteam_YC_got_M,Ateam_YC_got_M,Hteam_YC_opp_M,Ateam_YC_opp_M,Hteam_RC_got_M,Ateam_RC_got_M,Hteam_RC_opp_M,Ateam_RC_opp_M,Hteam_CC_got_M,Ateam_CC_got_M,Hteam_CC_opp_M,Ateam_CC_opp_M,Hteam_F_got_M,Ateam_F_got_M,Hteam_F_opp_M,Ateam_F_opp_M,Hteam_SH_got_M,Ateam_SH_got_M,Hteam_SH_opp_M,Ateam_SH_opp_M,Hteam_SoT_got_M,Ateam_SoT_got_M,Hteam_SoT_opp_M,Ateam_SoT_opp_M,Diff_HT_G_M,Diff_AT_G_M,Diff_HT_YC_M,Diff_AT_YC_M,Diff_HT_RC_M,Diff_AT_RC_M,Diff_HT_CC_M,Diff_AT_CC_M,Diff_HT_F_M,Diff_AT_F_M,Diff_HT_SH_M,Diff_AT_SH_M,Diff_HT_SoT_M,Diff_AT_SoT_M,Diff_ov,Hteam_Lat,Hteam_Lon
50,1411818300,27-09-2014,Liverpool,Everton,D,L,L,L,W,W,L,L,D,W,D,6,0.189189,0.297297,0.216216,0.351351,0.243243,0.216216,0.216216,0.324324,0.000000,0.0,0.000000,0.000000,0.648649,0.675676,0.864865,0.486486,1.189189,1.162162,1.756757,1.918919,1.405405,1.162162,0.972973,1.027027,0.540541,0.513514,0.513514,0.405405,-0.027027,-0.054054,0.027027,-0.108108,0.000000,0.000000,-0.216216,0.189189,-0.567568,-0.756757,0.432432,0.135135,0.027027,0.108108,1.030864,53.430819,-2.960828
51,1411826400,27-09-2014,Crystal Palace,Leicester City,H,W,W,D,W,D,D,L,L,L,D,6,0.216216,0.243243,0.270270,0.216216,0.297297,0.162162,0.297297,0.162162,0.027027,0.0,0.000000,0.027027,0.513514,0.540541,1.135135,1.189189,1.972973,1.837838,1.459459,1.189189,0.891892,0.783784,1.324324,1.702703,0.459459,0.324324,0.675676,0.756757,-0.054054,0.027027,0.000000,0.000000,0.027027,-0.027027,-0.621622,-0.648649,0.513514,0.648649,-0.432432,-0.918919,-0.216216,-0.432432,1.984568,51.398333,-0.085556
52,1411826400,27-09-2014,Southampton,Queens Park Rangers,H,W,D,W,L,W,W,D,L,L,L,6,0.243243,0.081081,0.081081,0.297297,0.216216,0.135135,0.216216,0.243243,0.000000,0.0,0.027027,0.000000,0.675676,0.783784,0.432432,0.702703,1.810811,1.027027,1.432432,1.540541,1.135135,1.459459,0.891892,1.216216,0.621622,0.486486,0.486486,0.540541,0.162162,-0.216216,0.000000,-0.108108,-0.027027,0.000000,0.243243,0.081081,0.378378,-0.513514,0.243243,0.243243,0.135135,-0.054054,0.123457,50.905833,-1.391111


### - Weather data

Creating a new dataframe with all the weather stats for the specific location and specific day.

#### BE CAREFUL
The chunck of code below takes <b>30 minutes</b> to process. This because for every single line of the dataframe "core_data" will make an call an API

In [19]:
# Import libraries for API calls
import urllib.request
import json

# New empty dataframe
df = pd.DataFrame(None)

In [20]:
def get_weather(key, lat, lon , timestamp):
    
    # Url to call
    url = "https://api.darksky.net/forecast/"+ key + "/"+ str(lat) +","+ str(lon) +","+ str(timestamp)+ "?exclude=currently,minutely,hourly,flags"
    req = urllib.request.Request(url)
    
    # Download data
    data = urllib.request.urlopen(req).read()
    
    # Convert data in json format, readable in utf-8
    data = json.loads(data.decode("utf-8"))
    # Using only daily data
    data = data["daily"]
    
    # Appendinfd data to the dataframe
    for v in data.values():
        for i in v:
            match = pd.DataFrame.from_dict(i, orient='index').T
    return match

In [21]:
# API key lists

API_key_list = ["c26d131ccc87bea5fe112e69f88cd8e0",
                "e361378e28e56671be3716420b892956",
                "1bf1fa75f88dc023379885d086d6fc67",
                "05bff9d0f09e7667f02a302b07d5c3a8",
                "42457ff003685128ccf352ec80f8cf15",
                "483e4b2af2a109646aa13baae7a0fcc9",
                "3c6bc9fb3aa3266464776d5be3e47aa2",
                "efee5540c38a8917922101cb257bab6f",
                "c51f8e88f820c58b58cc1b2bdd6b5d3d",
                "49276a3a91c106dc4a5c8205f9a10976",
                "d87ff68e722a913be0621c4c3d4ceedf",
                "344a9664de86133d5fe293d3538b47ee",
                "22d78f52e883c45e53dfd3750147c665",
                "a1c26a29cda4f609d9c5a87392c1f19e",
                "40107fd03d4d7d9785f87afacba21bf8",
                "31e4768b459491b9d341733a8be57fd5",
                "64013314a120aec059fde9c3aeeacb48",
                "7ac8891febdfee749f645300cc8d4438",
                "4bef002527d9ba5cd976d91f3fe6847e",
                "f1daea4e318d9749d338a23c4c9e4283",
                "58aba5c24fe603937f4602d627f3ea57",
                "c89e2f8fe3b74aeecec9ae02f24f8357",
                "55a37f923e993e1156f4eb1652dc9283",
                "7c384d8d4882d98289884abf329efa0c",
                "97c83e3547e67d2afb236df9f0350b83",]

In [22]:
# Number of times that API has been used
counter = 0 
# API index to be used
n = 0
for timestamp, lat, lon in zip(core_data["timestamp"], core_data["Hteam_Lat"], core_data["Hteam_Lon"]):
    # Get the API key
    key = API_key_list[n]
    
    if counter < 999:
        key = API_key_list[n]
        # Dowload the data
        match = get_weather(key, lat, lon, timestamp)
        # Append to the dataframe
        df = df.append(match, ignore_index = True)
        counter += 1
    
    else:
        # Reset the counter
        counter = 0
        # Changing API key --> using the next one
        n +=1
        key = API_key_list[n]
        # Dowload the data
        match = get_weather(key, lat, lon, timestamp)
        # Append to the dataframe
        df = df.append(match, ignore_index = True)
        counter += 1
data_weather = df.copy()

### - Last Utilities
Merging game stats and weather stats, dealing with NaN values and creating dummy features.

In [23]:
# Merge the dataset
dataset = pd.concat([core_data.reset_index(drop = True),
                     data_weather.reset_index(drop = True)],axis = 1)

# In the pressure column we have 2 missing values, so we fill them with the median
pressure_median = dataset.pressure.median()
dataset["pressure"].fillna(value = pressure_median, inplace = True)

# In the cloudCover column we have 12 missing values, so we fill them with zero
dataset["cloudCover"].fillna(value = 0, inplace = True)

# In the precipType column we have some missing values, so we fill them with "None"
dataset["precipType"].fillna(value = "None", inplace = True)

# Transform in dummies
to_dummy = ["Hteam_Bef1","Ateam_Bef1","Hteam_Bef2","Ateam_Bef2","Hteam_Bef3","Ateam_Bef3","Hteam_Bef4","Ateam_Bef4","Hteam_Bef5","Ateam_Bef5","precipType"]
dataset = pd.get_dummies(dataset, prefix = to_dummy, columns = to_dummy)

dataset.head(3)

,timestamp,date_GMT,home_team_name,away_team_name,Result,MW,Hteam_G_got_M,Ateam_G_got_M,Hteam_G_opp_M,Ateam_G_opp_M,Hteam_YC_got_M,Ateam_YC_got_M,Hteam_YC_opp_M,Ateam_YC_opp_M,Hteam_RC_got_M,Ateam_RC_got_M,Hteam_RC_opp_M,Ateam_RC_opp_M,Hteam_CC_got_M,Ateam_CC_got_M,Hteam_CC_opp_M,Ateam_CC_opp_M,Hteam_F_got_M,Ateam_F_got_M,Hteam_F_opp_M,Ateam_F_opp_M,Hteam_SH_got_M,Ateam_SH_got_M,Hteam_SH_opp_M,Ateam_SH_opp_M,Hteam_SoT_got_M,Ateam_SoT_got_M,Hteam_SoT_opp_M,Ateam_SoT_opp_M,Diff_HT_G_M,Diff_AT_G_M,Diff_HT_YC_M,Diff_AT_YC_M,Diff_HT_RC_M,Diff_AT_RC_M,Diff_HT_CC_M,Diff_AT_CC_M,Diff_HT_F_M,Diff_AT_F_M,Diff_HT_SH_M,Diff_AT_SH_M,Diff_HT_SoT_M,Diff_AT_SoT_M,Diff_ov,Hteam_Lat,Hteam_Lon,apparentTemperatureHigh,apparentTemperatureHighTime,apparentTemperatureLow,apparentTemperatureLowTime,apparentTemperatureMax,apparentTemperatureMaxTime,apparentTemperatureMin,apparentTemperatureMinTime,cloudCover,dewPoint,humidity,icon,moonPhase,ozone,precipAccumulation,precipIntensity,precipIntensityMax,precipIntensityMaxTime,precipProbability,pressure,summary,sunriseTime,sunsetTime,temperatureHigh,temperatureHighTime,temperatureLow,temperatureLowTime,temperatureMax,temperatureMaxTime,temperatureMin,temperatureMinTime,time,uvIndex,uvIndexTime,visibility,windBearing,windGust,windGustTime,windSpeed,Hteam_Bef1_D,Hteam_Bef1_L,Hteam_Bef1_W,Ateam_Bef1_D,Ateam_Bef1_L,Ateam_Bef1_W,Hteam_Bef2_D,Hteam_Bef2_L,Hteam_Bef2_W,Ateam_Bef2_D,Ateam_Bef2_L,Ateam_Bef2_W,Hteam_Bef3_D,Hteam_Bef3_L,Hteam_Bef3_W,Ateam_Bef3_D,Ateam_Bef3_L,Ateam_Bef3_W,Hteam_Bef4_D,Hteam_Bef4_L,Hteam_Bef4_W,Ateam_Bef4_D,Ateam_Bef4_L,Ateam_Bef4_W,Hteam_Bef5_D,Hteam_Bef5_L,Hteam_Bef5_W,Ateam_Bef5_D,Ateam_Bef5_L,Ateam_Bef5_W,precipType_None,precipType_rain,precipType_snow
0,1411818300,27-09-2014,Liverpool,Everton,D,6,0.189189,0.297297,0.216216,0.351351,0.243243,0.216216,0.216216,0.324324,0.000000,0.0,0.000000,0.000000,0.648649,0.675676,0.864865,0.486486,1.189189,1.162162,1.756757,1.918919,1.405405,1.162162,0.972973,1.027027,0.540541,0.513514,0.513514,0.405405,-0.027027,-0.054054,0.027027,-0.108108,0.000000,0.000000,-0.216216,0.189189,-0.567568,-0.756757,0.432432,0.135135,0.027027,0.108108,1.030864,53.430819,-2.960828,64.29,1411830900,55.96,1411862160,64.29,1411830900,45.39,1411800000,0.48,48.69,0.77,rain,0.11,NaN,NaN,0.0357,0.0573,1411792620,0.41,1025.2,Possible light rain in the morning.,1411798080,1411840860,64.79,1411830900,55.47,1411862160,64.79,1411830900,45.52,1411798440,1411772400,3,1411819260,6.132,221,8.02,1411825800,3.99,0,1,0,0,1,0,0,1,0,0,0,1,0,0,1,0,1,0,0,1,0,1,0,0,0,0,1,1,0,0,0,1,0
1,1411826400,27-09-2014,Crystal Palace,Leicester City,H,6,0.216216,0.243243,0.270270,0.216216,0.297297,0.162162,0.297297,0.162162,0.027027,0.0,0.000000,0.027027,0.513514,0.540541,1.135135,1.189189,1.972973,1.837838,1.459459,1.189189,0.891892,0.783784,1.324324,1.702703,0.459459,0.324324,0.675676,0.756757,-0.054054,0.027027,0.000000,0.000000,0.027027,-0.027027,-0.621622,-0.648649,0.513514,0.648649,-0.432432,-0.918919,-0.216216,-0.432432,1.984568,51.398333,-0.085556,69.57,1411824780,56.33,1411885920,69.57,1411824780,50.99,1411791180,0.69,53.62,0.78,partly-cloudy-day,0.11,NaN,NaN,0,0,NaN,0,1026.3,Mostly cloudy throughout the day.,1411797360,1411840140,70.07,1411824780,55.84,1411885920,70.07,1411824780,50.5,1411791180,1411772400,3,1411818840,6.085,131,3.96,1411826760,0.82,0,0,1,0,0,1,1,0,0,0,0,1,1,0,0,1,0,0,0,1,0,0,1,0,0,1,0,1,0,0,1,0,0
2,1411826400,27-09-2014,Southampton,Queens Park Rangers,H,6,0.243243,0.081081,0.081081,0.297297,0.216216,0.135135,0.216216,0.243243,0.000000,0.0,0.027027,0.000000,0.675676,0.783784,0.432432,0.702703,1.810811,1.027027,1.432432,1.540541,1.135135,1.459459,0.891892,1.216216,0.621622,0.486486,0.486486,0.540541,0.162162,-0.216216,0.000000,-0.108108,-0.027027,0.000000,0.243243,0.081081,0.378378,-0.513514,0.243243,0.243243,0.135135,-0.054054,0.123457,50.905833,-1.391111,69.52,1411819380,52.08,1411883520,69.52,1411819380,52.33,1411790160,0.65,54.86,0.85,partly-cloudy-day,0.11,NaN,NaN,0,0,NaN

In [24]:
# Reorder columns
col = ['MW','Result',
       'home_team_name','away_team_name',
       'Hteam_G_got_M','Ateam_G_got_M',
       'Hteam_G_opp_M','Ateam_G_opp_M',
       'Hteam_CC_got_M','Ateam_CC_got_M',
       'Hteam_CC_opp_M','Ateam_CC_opp_M',
       'Hteam_F_got_M','Ateam_F_got_M',
       'Hteam_F_opp_M','Ateam_F_opp_M',
       'Hteam_YC_got_M','Ateam_YC_got_M',
       'Hteam_YC_opp_M','Ateam_YC_opp_M',
       'Hteam_RC_got_M','Ateam_RC_got_M',
       'Hteam_RC_opp_M','Ateam_RC_opp_M',
       'Hteam_SH_got_M','Ateam_SH_got_M',
       'Hteam_SH_opp_M','Ateam_SH_opp_M',
       'Hteam_SoT_got_M','Ateam_SoT_got_M',
       'Hteam_SoT_opp_M','Ateam_SoT_opp_M',
       'Diff_HT_G_M','Diff_AT_G_M',
       'Diff_HT_CC_M','Diff_AT_CC_M',
       'Diff_HT_F_M','Diff_AT_F_M',
       'Diff_HT_RC_M','Diff_AT_RC_M',
       'Diff_HT_SH_M','Diff_AT_SH_M',
       'Diff_HT_SoT_M','Diff_AT_SoT_M',
       'Diff_HT_YC_M','Diff_AT_YC_M',
       'Diff_ov',
       'Hteam_Bef1_W','Hteam_Bef1_D','Hteam_Bef1_L',
       'Ateam_Bef1_W','Ateam_Bef1_D','Ateam_Bef1_L',
       'Hteam_Bef2_W','Hteam_Bef2_D','Hteam_Bef2_L',
       'Ateam_Bef2_W','Ateam_Bef2_D','Ateam_Bef2_L',
       'Hteam_Bef3_W','Hteam_Bef3_D', 'Hteam_Bef3_L',
       'Ateam_Bef3_W','Ateam_Bef3_D', 'Ateam_Bef3_L',
       'Hteam_Bef4_W','Hteam_Bef4_D','Hteam_Bef4_L',
       'Ateam_Bef4_W','Ateam_Bef4_D','Ateam_Bef4_L',
       'Hteam_Bef5_W','Hteam_Bef5_D', 'Hteam_Bef5_L',
       'Ateam_Bef5_W','Ateam_Bef5_D', 'Ateam_Bef5_L',
       'apparentTemperatureHigh','apparentTemperatureLow','apparentTemperatureMax', 'apparentTemperatureMin',
       'cloudCover','dewPoint','humidity','moonPhase', 'pressure',
       'precipType_None','precipType_rain', 'precipType_snow','precipIntensity','precipIntensityMax','precipProbability',
       'temperatureHigh','temperatureLow','temperatureMax','temperatureMin',
       'uvIndex','visibility',
       'windBearing','windGust','windSpeed']

# Keep best columns
dataset = dataset[col]

dataset.head(3)

,MW,Result,home_team_name,away_team_name,Hteam_G_got_M,Ateam_G_got_M,Hteam_G_opp_M,Ateam_G_opp_M,Hteam_CC_got_M,Ateam_CC_got_M,Hteam_CC_opp_M,Ateam_CC_opp_M,Hteam_F_got_M,Ateam_F_got_M,Hteam_F_opp_M,Ateam_F_opp_M,Hteam_YC_got_M,Ateam_YC_got_M,Hteam_YC_opp_M,Ateam_YC_opp_M,Hteam_RC_got_M,Ateam_RC_got_M,Hteam_RC_opp_M,Ateam_RC_opp_M,Hteam_SH_got_M,Ateam_SH_got_M,Hteam_SH_opp_M,Ateam_SH_opp_M,Hteam_SoT_got_M,Ateam_SoT_got_M,Hteam_SoT_opp_M,Ateam_SoT_opp_M,Diff_HT_G_M,Diff_AT_G_M,Diff_HT_CC_M,Diff_AT_CC_M,Diff_HT_F_M,Diff_AT_F_M,Diff_HT_RC_M,Diff_AT_RC_M,Diff_HT_SH_M,Diff_AT_SH_M,Diff_HT_SoT_M,Diff_AT_SoT_M,Diff_HT_YC_M,Diff_AT_YC_M,Diff_ov,Hteam_Bef1_W,Hteam_Bef1_D,Hteam_Bef1_L,Ateam_Bef1_W,Ateam_Bef1_D,Ateam_Bef1_L,Hteam_Bef2_W,Hteam_Bef2_D,Hteam_Bef2_L,Ateam_Bef2_W,Ateam_Bef2_D,Ateam_Bef2_L,Hteam_Bef3_W,Hteam_Bef3_D,Hteam_Bef3_L,Ateam_Bef3_W,Ateam_Bef3_D,Ateam_Bef3_L,Hteam_Bef4_W,Hteam_Bef4_D,Hteam_Bef4_L,Ateam_Bef4_W,Ateam_Bef4_D,Ateam_Bef4_L,Hteam_Bef5_W,Hteam_Bef5_D,Hteam_Bef5_L,Ateam_Bef5_W,Ateam_Bef5_D,Ateam_Bef5_L,apparentTemperatureHigh,apparentTemperatureLow,apparentTemperatureMax,apparentTemperatureMin,cloudCover,dewPoint,humidity,moonPhase,pressure,precipType_None,precipType_rain,precipType_snow,precipIntensity,precipIntensityMax,precipProbability,temperatureHigh,temperatureLow,temperatureMax,temperatureMin,uvIndex,visibility,windBearing,windGust,windSpeed
0,6,D,Liverpool,Everton,0.189189,0.297297,0.216216,0.351351,0.648649,0.675676,0.864865,0.486486,1.189189,1.162162,1.756757,1.918919,0.243243,0.216216,0.216216,0.324324,0.000000,0.0,0.000000,0.000000,1.405405,1.162162,0.972973,1.027027,0.540541,0.513514,0.513514,0.405405,-0.027027,-0.054054,-0.216216,0.189189,-0.567568,-0.756757,0.000000,0.000000,0.432432,0.135135,0.027027,0.108108,0.027027,-0.108108,1.030864,0,0,1,0,0,1,0,0,1,1,0,0,1,0,0,0,0,1,0,0,1,0,1,0,1,0,0,0,1,0,64.29,55.96,64.29,45.39,0.48,48.69,0.77,0.11,1025.2,0,1,0,0.0357,0.0573,0.41,64.79,55.47,64.79,45.52,3,6.132,221,8.02,3.99
1,6,H,Crystal Palace,Leicester City,0.216216,0.243243,0.270270,0.216216,0.513514,0.540541,1.135135,1.189189,1.972973,1.837838,1.459459,1.189189,0.297297,0.162162,0.297297,0.162162,0.027027,0.0,0.000000,0.027027,0.891892,0.783784,1.324324,1.702703,0.459459,0.324324,0.675676,0.756757,-0.054054,0.027027,-0.621622,-0.648649,0.513514,0.648649,0.027027,-0.027027,-0.432432,-0.918919,-0.216216,-0.432432,0.000000,0.000000,1.984568,1,0,0,1,0,0,0,1,0,1,0,0,0,1,0,0,1,0,0,0,1,0,0,1,0,0,1,0,1,0,69.57,56.33,69.57,50.99,0.69,53.62,0.78,0.11,1026.3,1,0,0,0,0,0,70.07,55.84,70.07,50.5,3,6.085,131,3.96,0.82
2,6,H,Southampton,Queens Park Rangers,0.243243,0.081081,0.081081,0.297297,0.675676,0.783784,0.432432,0.702703,1.810811,1.027027,1.432432,1.540541,0.216216,0.135135,0.216216,0.243243,0.000000,0.0,0.027027,0.000000,1.135135,1.459459,0.891892,1.216216,0.621622,0.486486,0.486486,0.540541,0.162162,-0.216216,0.243243,0.081081,0.378378,-0.513514,-0.027027,0.000000,0.243243,0.243243,0.135135,-0.054054,0.000000,-0.108108,0.123457,1,0,0,0,1,0,1,0,0,0,0,1,1,0,0,1,0,0,0,1,0,0,0,1,0,0,1,0,0,1,69.52,52.08,69.52,52.33,0.65,54.86,0.85,0.11,1026.0,1,0,0,0,0,0,70.02,51.59,70.02,51.84,3,6.164,118,6.51,1.18


### Exporting the Dataframe
Exporting the dataframe to a .csv file

In [25]:
# Export the file
dataset.to_csv("dataset/Other/" + "dataset.csv")